In [36]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [37]:
quimica_template = ChatPromptTemplate.from_template("""Você é um químico muito experiente.
Você é excelente em responder perguntas sobre química de forma clara e objetiva.
Você tem um grande entendimento sobre reações químicas, elementos, compostos,
e a relação entre a estrutura molecular e as propriedades dos materiais.
Quando você não sabe a resposta para uma pergunta, você admite que não sabe.

Aqui está uma pergunta: {input}""")

geografia_template = ChatPromptTemplate.from_template("""Você é um geógrafo muito bem informado.
Você tem um vasto conhecimento sobre os processos naturais, geografia humana, 
clima, relevo e interações entre o ambiente e a sociedade.
Você é habilidoso em explicar como fatores físicos e humanos afetam
o mundo ao nosso redor.

Aqui está uma pergunta: {input}""")

biologia_template = ChatPromptTemplate.from_template("""Você é um biólogo muito capacitado.
Você tem um grande conhecimento sobre os seres vivos, suas estruturas, funções
e a interação entre diferentes organismos e seus ambientes.
Você é excelente em explicar conceitos de biologia de maneira clara,
tanto para iniciantes quanto para estudantes avançados.

Aqui está uma pergunta: {input}""")


In [38]:
prompt_infos = [
    {
        "name": "Química",
        "description": "Ideal para responder pergunta de química",
        "prompt_template": quimica_template
    },
     {
        "name": "Geografia",
        "description": "Ideal para responder pergunta de geografia",
        "prompt_template": geografia_template
    },
      {
        "name": "Biologia",
        "description": "Ideal para responder pergunta de biologia",
        "prompt_template": biologia_template
    },
]

In [39]:
chat = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [40]:
chains_destino = {}
for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info["prompt_template"], verbose=True)
    chains_destino[info["name"]] = chain
    
chains_destino

{'Química': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Você é um químico muito experiente.\nVocê é excelente em responder perguntas sobre química de forma clara e objetiva.\nVocê tem um grande entendimento sobre reações químicas, elementos, compostos,\ne a relação entre a estrutura molecular e as propriedades dos materiais.\nQuando você não sabe a resposta para uma pergunta, você admite que não sabe.\n\nAqui está uma pergunta: {input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000021ABF7423F0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000021ABF745940>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='')),
 'Geografia': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=Prom

In [42]:
destinos = [f'{p["name"]}: {p["description"]}' for p in prompt_infos]
destinos_str = "\n".join(destinos)
print(destinos_str)

Química: Ideal para responder pergunta de química
Geografia: Ideal para responder pergunta de geografia
Biologia: Ideal para responder pergunta de biologia


In [49]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinos_str
)

print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Química: Ideal para responder pergunta de química
Geografia: Ideal para responder pergunta de geograf

In [50]:
router_template = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(chat, router_template, verbose=True)

In [51]:
default_prompt = ChatPromptTemplate.from_template(f"{input}")
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destino,
    default_chain=default_chain,
    verbose=True
)

In [52]:
chain.invoke({"input": "O que é o El Niño?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Geografia: {'input': 'O que é o El Niño?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um geógrafo muito bem informado.
Você tem um vasto conhecimento sobre os processos naturais, geografia humana, 
clima, relevo e interações entre o ambiente e a sociedade.
Você é habilidoso em explicar como fatores físicos e humanos afetam
o mundo ao nosso redor.

Aqui está uma pergunta: O que é o El Niño?

> Finished chain.

> Finished chain.


{'input': 'O que é o El Niño?',
 'text': 'O El Niño é um fenômeno climático que ocorre de forma periódica no Oceano Pacífico, caracterizado pelo aquecimento anormal das águas superficiais do oceano na região equatorial. Esse aquecimento altera os padrões de vento e de chuva em diversas regiões do mundo, podendo causar secas, inundações e mudanças drásticas no clima.\n\nDurante um evento de El Niño, as águas quentes do oceano deslocam-se em direção à costa oeste da América do Sul, provocando chuvas intensas e enchentes nessa região. Ao mesmo tempo, áreas como Austrália, Indonésia e Índia podem enfrentar secas severas devido à mudança nos padrões de vento e de precipitação.\n\nO El Niño também pode afetar a pesca, a agricultura, a biodiversidade e a economia de diversos países, gerando impactos sociais, econômicos e ambientais significativos. É importante monitorar e compreender esse fenômeno para melhorar a previsão de eventos climáticos extremos e mitigar seus impactos.'}

In [53]:
chain.invoke({"input": "Para que serve a tabela periódica?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Química: {'input': 'Para que serve a tabela periódica?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um químico muito experiente.
Você é excelente em responder perguntas sobre química de forma clara e objetiva.
Você tem um grande entendimento sobre reações químicas, elementos, compostos,
e a relação entre a estrutura molecular e as propriedades dos materiais.
Quando você não sabe a resposta para uma pergunta, você admite que não sabe.

Aqui está uma pergunta: Para que serve a tabela periódica?

> Finished chain.

> Finished chain.


{'input': 'Para que serve a tabela periódica?',
 'text': 'A tabela periódica serve para organizar e classificar os elementos químicos de acordo com suas propriedades e características. Ela fornece informações importantes sobre os elementos, como número atômico, massa atômica, configuração eletrônica, propriedades físicas e químicas, entre outras. A tabela periódica é uma ferramenta fundamental para a química, pois permite prever o comportamento dos elementos e facilita a compreensão das relações entre eles.'}

In [56]:
chain.invoke({"input": "Para que serve a fotosíntese?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Biologia: {'input': 'Para que serve a fotossíntese?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um biólogo muito capacitado.
Você tem um grande conhecimento sobre os seres vivos, suas estruturas, funções
e a interação entre diferentes organismos e seus ambientes.
Você é excelente em explicar conceitos de biologia de maneira clara,
tanto para iniciantes quanto para estudantes avançados.

Aqui está uma pergunta: Para que serve a fotossíntese?

> Finished chain.

> Finished chain.


{'input': 'Para que serve a fotossíntese?',
 'text': 'A fotossíntese é um processo fundamental realizado pelas plantas, algas e algumas bactérias, que convertem a energia solar em energia química, na forma de glicose. Esse processo é essencial para a vida na Terra, pois fornece o oxigênio que respiramos e é a base da cadeia alimentar, já que as plantas são a principal fonte de alimento para a maioria dos seres vivos. Além disso, a fotossíntese ajuda a regular o clima, absorvendo dióxido de carbono da atmosfera e liberando oxigênio. Em resumo, a fotossíntese é responsável por produzir o alimento e o oxigênio que sustentam a vida no planeta.'}